In [10]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import xmltodict
import json

In [2]:
df = pd.read_excel('../data/grid.xlsx')
df.head()

,Fecha,N° Boletín,Título,Estado
0,22/09/2020,13800-07,"Modifica la Carta Fundamental, para establecer...",En tramitación
1,17/09/2020,13798-07,"Modifica la Carta Fundamental, para exigir el ...",En tramitación
2,16/09/2020,13796-07,"Modifica la Carta Fundamental, para eliminar l...",En tramitación
3,16/09/2020,13797-07,"Modifica la Carta Fundamental, para determinar...",En tramitación
4,16/09/2020,13792-07,Proyecto de reforma constitucional que facilit...,En tramitación


In [4]:
pd.unique(df['Estado'].values.ravel())

array(['En tramitación', 'Publicado: Ley N° 21.261',
       'Publicado: Ley N° 21.257', 'Publicado: Ley N° 21.248',
       'Publicado: Ley N° 21.253', 'Publicado: Ley N° 21.237',
       'Publicado: Ley N° 21.221', 'Publicado: Ley N° 21.219',
       'Publicado: Ley N° 21.216', 'Rechazado',
       'Publicado: Ley N° 21.200', 'Publicado: Ley N° 21233', 'Retirado',
       'Publicado: Ley N° 21.011', 'Archivado',
       'Publicado: Ley N° 20.990', 'Publicado: Ley N° 20.870',
       'Publicado: Ley N° 21.096', 'Publicado: Ley N° 21.233',
       'Publicado: Ley Nº 20.725', 'Publicado: Ley Nº 20.748',
       'Publicado: Ley Nº 20.710', 'Publicado: Ley N° 20854',
       'Publicado: Ley N° 21.238', 'Publicado: Ley N° 20.515',
       'Publicado: Ley Nº 20.573', 'Publicado: Ley N° 20.516',
       'Publicado: Ley Nº 20.503', 'Publicado: Ley Nº 20.193',
       'Publicado: Ley Nº 20.162', 'Publicado: Ley Nº 20.390',
       'Publicado: Ley Nº 20.352', 'Publicado: Ley Nº 20.050',
       'Publicado: Ley

In [28]:
def get_url(string):
    return "https://www.senado.cl/wspublico/tramitacion.php?boletin=" + string

campos = [['autores', 'autor'], ['comparados', 'comparado'], ['materias', 'materia'],
        ['indicaciones', 'indicacion', 'FECHA'], ['informes', 'informe', 'FECHAINFORME'],
        ['oficios', 'oficio', 'FECHA'], ['tramitacion', 'tramite', 'FECHA'],
        ['urgencias', 'urgencia', 'FECHAINGRESO'], ['votaciones', 'votacion', 'FECHA']]

new_json = []

for i in range(df.shape[0]-1, -1, -1):

    boletin = df.iloc[i]['N° Boletín'].split('-')[0]
    url = get_url(boletin)

    file = urlopen(url)
    xmldata = file.read()
    file.close()

    data = xmltodict.parse(xmldata, force_list=('indicacion', 'informe', 'oficio', 'tramite',
                                               'urgencia', 'votacion'))
    json_data = json.dumps(data, ensure_ascii=False)
    projects = json.loads(json_data)

    proyecto = projects['proyectos']['proyecto']
    json_project = {}
    json_project['descripcion'] = proyecto['descripcion']
    for campo in campos:
        if proyecto[campo[0]] is not None:
            json_project[campo[0]] = proyecto[campo[0]][campo[1]]
            if campo[0] == 'tramitacion':
                arr = []
                for i,tramite in enumerate(json_project[campo[0]]):
                    if i == 0:
                        obj = {}
                        camara = tramite['CAMARATRAMITE']
                        fecha_inicio = tramite['FECHA']
                    if tramite['CAMARATRAMITE'] != camara:
                        fecha_termino = tramite['FECHA']
                        obj['camara'] = camara
                        obj['inicio'] = fecha_inicio
                        obj['termino'] = fecha_termino
                        arr.append(obj)

                        obj = {}
                        camara = tramite['CAMARATRAMITE']
                        fecha_inicio = tramite['FECHA']
                    if i == len(json_project[campo[0]]) - 1:
                        fecha_termino = tramite['FECHA']
                        obj['camara'] = camara
                        obj['inicio'] = fecha_inicio
                        obj['termino'] = fecha_termino
                        arr.append(obj)
                json_project['camaras'] = arr
        else:
            json_project[campo[0]] = None
            if campo[0] == 'tramitacion':
                json_project['camaras'] = None
            
    new_json.append(json_project)

In [29]:
with open("../data/reformas.json", "w") as json_file:
    this_file = json.dumps(new_json, ensure_ascii=False)
    json_file.write(this_file)